In [10]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse
from collections import deque
import pprint
import os # 拡張子をチェックするために os.path を使用


START_URL = "https://www.musashino-u.ac.jp/"#開始地点となるURL
BASE_DOMAIN = "musashino-u.ac.jp"#クロール対象のドメイン
WAIT_TIME = 0.8 #負荷軽減のための待機時間（秒）
EXCLUDE_EXTENSIONS = [
    #画像
    '.png', '.jpg', '.jpeg', '.gif', '.svg', '.bmp', '.webp',
    #ドキュメント
    '.pdf', '.doc', '.docx', '.xls', '.xlsx', '.ppt', '.pptx',
    #圧縮ファイル
    '.zip', '.rar', '.gz',
    #メディア
    '.mp4', '.mov', '.avi', '.mp3', '.wav',
    #その他
    '.css', '.js'
] #クロール対象から除外するファイルの拡張子の設定
def crawl_sitemap_v2():  
    sitemap = {} #サイトマップを格納する辞書を設定
    queue = deque([START_URL]) #これから訪問するURLを管理
    visited_urls = {START_URL} #訪問済みURLのセット
    TARGET_NETLOC = urlparse(START_URL).netloc  # 開始URLのネットロケーション部分を取得
    headers = {
        "User-Agent": "MySitemapScraper/1.0 (Contact: your-email@example.com)"
    }
    while queue :
        current_url = queue.popleft()

        print(f"[{len(sitemap) + 1}] 処理中: {current_url}", end="")

        try:
            time.sleep(WAIT_TIME)#負荷軽減のための待機時間(1秒)
            response = requests.get(current_url, headers=headers, timeout=10)
            if response.status_code != 200:
                print(" -> スキップ (Status Code)") #ステータスコードが200でない場合はスキップ
                continue 
            if 'text/html' not in response.headers.get('Content-Type', ''):#コンテンツタイプがHTML/テキストでない場合はスキップ
                print(" -> スキップ (HTMLではありません)")
                continue
                
            response.encoding = response.apparent_encoding
            
            soup = BeautifulSoup(response.text, "html.parser")

            #ページごとの<title>を取得
            title_tag = soup.find("title")
            title = "タイトルなし" 
            if title_tag and title_tag.string:
                title = title_tag.string.strip()
            
            sitemap[current_url] = title #辞書型変数に格納
            print(f" -> {title}")

            links = soup.find_all("a", href=True) #同一ドメインの全リンクを巡る(find_allでaタグを抽出)

            for link in links:
                href = link.get("href")
                absolute_url = urljoin(current_url, href) #相対URLを絶対URLに変換
                parsed_url = urlparse(absolute_url) #URLを正規化
                clean_url = parsed_url._replace(query="", fragment="").geturl()#クエリとフラグメントを除去

                path = parsed_url.path #(a)パスから拡張子を取得
                ext = os.path.splitext(path)[1].lower() #os.path.splitext で拡張子を小文字に変換

                #チェック作業
                is_valid = (
                    parsed_url.netloc == TARGET_NETLOC and     #対象ドメインと完全に一致するか (サブドメインを除外)
                    parsed_url.scheme in ["http", "https"] and #HTTP/HTTPSプロトコルか
                    clean_url not in visited_urls and         #未訪問のURLか
                    ext not in EXCLUDE_EXTENSIONS             #除外対象の拡張子でないか
                )


                if is_valid:
                    visited_urls.add(clean_url) #訪問済みリストに追加
                    queue.append(clean_url) #キューに追加

        except requests.exceptions.RequestException as e: #リクエスト例外処理
            print(f" -> エラー ( {e} )")
        except Exception as e:
            print(f" -> 不明なエラー ( {e} )")

    print("\nクロールが完了しました。") #クロール完了メッセージ
    return sitemap


#メイン処理
if __name__ == "__main__":
    #クローラーの実行
    site_map_data = crawl_sitemap_v2()
    print("\n--- サイトマップ（辞書型） ---") #辞書型変数をprint() で表示
    pprint.pprint(site_map_data)
    print(f"\n合計 {len(site_map_data)} ページを検出しました。")

[1] 処理中: https://www.musashino-u.ac.jp/ -> 武蔵野大学
[2] 処理中: https://www.musashino-u.ac.jp/access.html -> 交通アクセス | 武蔵野大学
[3] 処理中: https://www.musashino-u.ac.jp/admission/request.html -> 資料請求 | 入試情報 | 武蔵野大学
[4] 処理中: https://www.musashino-u.ac.jp/contact.html -> お問い合わせ | 武蔵野大学
[5] 処理中: https://www.musashino-u.ac.jp/prospective-students.html -> 武蔵野大学で学びたい方 | 武蔵野大学
[6] 処理中: https://www.musashino-u.ac.jp/students.html -> 在学生の方 | 武蔵野大学
[7] 処理中: https://www.musashino-u.ac.jp/alumni.html -> 卒業生の方 | 武蔵野大学
[8] 処理中: https://www.musashino-u.ac.jp/parents.html -> 保護者の方 | 武蔵野大学
[9] 処理中: https://www.musashino-u.ac.jp/business.html -> 企業・研究者の方 | 武蔵野大学
[10] 処理中: https://www.musashino-u.ac.jp/guide/ -> 大学案内 | 武蔵野大学
[11] 処理中: https://www.musashino-u.ac.jp/guide/profile/ -> 大学紹介 | 大学案内 | 武蔵野大学
[12] 処理中: https://www.musashino-u.ac.jp/guide/activities/ -> 大学の取り組み | 大学案内 | 武蔵野大学
[13] 処理中: https://www.musashino-u.ac.jp/guide/campus/ -> キャンパス | 大学案内 | 武蔵野大学
[14] 処理中: https://www.musashino-u.ac.jp/guide/facility/ 